# Phase 1: Research & Setup

Objective: Define the model scope, set up the development environment, and prepare data sources.

## ✅ Define System Scope

### Identify agent types

All agents have one goal which is to maximise their profit.

- **Consumer**:
    - Type: None
    - Attributes: persona, initial resources, energy mix, energy needs, buying price (from prosumers), distance from grid ?
    - Actions: do nothing, buy from utility, buy from local grid (prosumer)
<br> 
<br>

- **Prosumer (inherited from Consumer)**: consumer that also produces energy locally and can either sell it on the grid or use it for its personnal consumption 
    - Type: photovoltaic (PV)
    - Attributes: inherited from consumers, production level, max capacity of production, electricity storage level, selling price
    - Objects: maintainance rate of the PV, lifetime of the PV
    - Actions: inherited from consumer, change sell price on the local grid, change volume sold on the local grid, disconnect from local grid
<br> 
<br>

- **Energy producer**: generate energy and determine their production levels, facility upgrades, and pricing strategies
    - Type: oil, renewables
    - State: connected energy suppliers/utilities
    - Attributes: persona, initial resources, volume of production, max production capacity, production costs, price to sell energy to utility
    - Actions: change price to sell energy to utility, invest money to upgrade facility max capacity
<br> 
<br>

- **Energy utility**: purchase energy from producers and sell it to consumers
    - Type: eco-friendly, balanced, greedy
    - Attributes: persona, initial resources, selling price to consumer, volume of energy to deal, regulatory constraint (% of renewable to buy)
    - Actions: change price to sell energy to consumer, select energy supplier company, risk hedging by negotiating future contracts
<br> 
<br>

- ***Regulators***: dynamic regulation in reaction to market failures
    - Type: None
    - Attributes: persona
    - Actions: Penalties & incentives (fines on price gouging or reward green energy investments)


---

### Define market dynamics:

- Scenario 1: 
    -   Auction-based energy exchange for local grid
    -   Contracts with secured price for utility supplier
- Scenario 2: 
    - Auction-based trading for everything


---

### Select LLM use cases: 

- Consumer:
    - Decides wether to buy energy from utility or from the grid
    - Decides from which utility / grid node buy energy

- Prosumer (inherited from Consumer):
    - Decides wether to sell or consume energy produced

- Energy producer:
    - Determines energy production levels and selling prices
    - Decides whether to upgrade facilities to increase production limits (considering the associated costs)


- Utility:
    - Decides energy purchase amounts and consumer pricing to meet consumer demand
    - Randomly assigned one of three personas: [environmentally conscious, greedy, depressed]

- *Regulator*:
    - Decides wether to impose a policy to regulate the market

In [31]:
from mesa import Agent
import numpy as np

PRODUCER_TYPE = [
    "OilAndGas",
    "RenewableEnergy",]

UTILITY_PERSONA = [
    "environmentally conscious",
    "greedy",
    "depressed"    
]


class EnergyMarketAgent(Agent):
    """Base class for all agents in the energy market."""
    def __init__(self, unique_id, model, persona, initial_resources):
        super().__init__(unique_id, model)
        self.persona = persona
        self.resources = initial_resources
        self.profit = 0

    def step(self):
        pass

class ConsumerAgent(EnergyMarketAgent):
    """Consumer agent that purchases energy."""
    def __init__(self, unique_id, model, persona, initial_resources, energy_needs):
        super().__init__(unique_id, model, persona, initial_resources)
        # Initialize parameters for consumer
        self.energy_mix: dict = {}
        self.energy_needs: int = energy_needs
        self.buying_price = 0
    
    def step(self):
        # Decision making for buying energy
        self.decide_energy_source()

    def decide_energy_source(self):
        """LLM decision tio do nothing, buy from main grid or buy from local grid."""
        # TODO: Implement LLM-based decision making for energy source selection
        pass

class ProsumerAgent(ConsumerAgent):
    """Prosumer agent that can both consume and produce energy."""
    def __init__(self, unique_id, model, persona, initial_resources, energy_needs, max_capacity):
        super().__init__(unique_id, model, persona, initial_resources, energy_needs)
        # Initialize parameters for prosumer
        self.production_level = 0
        self.max_capacity = max_capacity
        self.storage_level = 0
        self.buying_price = 0
        self.selling_price = 0
        
    def step(self):
        super().step()
        self.decide_production_allocation()

    def decide_production_allocation():
        """LLM decision local grid strategy."""
        # TODO: Implement LLM-based decision making for production allocation
        pass

class EnergyProducerAgent(EnergyMarketAgent):
    """Energy producer agent that generates and sells energy."""
    def __init__(self, unique_id, model, persona, initial_resources, 
                 production_type, max_capacity, production_costs):
        super().__init__(unique_id, model, persona, initial_resources)
        self.production_type = production_type if production_type else str(np.random.choice(PRODUCER_TYPE))
        self.max_capacity = max_capacity
        self.production_costs = production_costs
        self.production_level = 0
        self.selling_price = 0
        
    def step(self):
        self.determine_production_strategy()
        
    def determine_production_strategy(self):
        # TODO: Implement LLM-based decision making for production strategy
        pass

class UtilityAgent(EnergyMarketAgent):
    """Utility agent that buys from producers and sells to consumers."""
    def __init__(self, unique_id, model, persona, initial_resources, 
                 utility_type, renewable_quota):
        super().__init__(unique_id, model, persona, initial_resources)
        self.utility_type = utility_type if utility_type else str(np.random.choice(UTILITY_PERSONA))
        self.renewable_quota = renewable_quota
        self.selling_price = 0
        self.energy_volume = 0
        
    def step(self):
        self.determine_market_strategy()
        
    def determine_market_strategy(self):
        # TODO: Implement LLM-based decision making for market strategy
        pass

class RegulatorAgent(EnergyMarketAgent):
    """Regulator agent that oversees market dynamics."""
    def __init__(self, unique_id, model, persona):
        super().__init__(unique_id, model, persona, float('inf'))

    def step(self):
        self.evaluate_market_conditions()
        
    def evaluate_market_conditions(self):
        # TODO: Implement LLM-based decision making for market regulation
        pass

In [32]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

class EnergyMarketModel(Model):
    """Energy market model with multiple agent types."""
    def __init__(self, 
                 num_consumers=4,
                 num_prosumers=2,
                 num_producers=2,
                 num_utilities=2,
                 width=20, 
                 height=20):
        super().__init__()
        self.num_consumers = num_consumers
        self.num_prosumers = num_prosumers
        self.num_producers = num_producers
        self.num_utilities = num_utilities
        
        # self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        
        # Create agents
        self.create_agents()
        
        # Data collection
        self.datacollector = DataCollector(
            model_reporters={
                "Average_Price": lambda m: np.mean([a.selling_price for a in m.schedule.agents 
                                                  if isinstance(a, (ProsumerAgent, EnergyProducerAgent, UtilityAgent))]),
                "Total_Production": lambda m: sum([a.production_level for a in m.schedule.agents 
                                                 if isinstance(a, (ProsumerAgent, EnergyProducerAgent))])
            },
            agent_reporters={
                "Profit": "profit",
                "Resources": "resources"
            }
        )
    
    def create_agents(self):
        # Create consumers
        consumers = []
        for i in range(self.num_consumers):
            consumer = ConsumerAgent(
                unique_id=f"consumer_{i}", 
                model=self, 
                persona="default",
                initial_resources=1000,
                energy_needs=np.random.uniform(50, 200))
            # self.schedule.add(consumer)
            consumers.append(consumer)
            
        # Create prosumers
        for i in range(self.num_prosumers):
            prosumer = ProsumerAgent(unique_id=f"prosumer_{i}", 
                                model=self, 
                              persona="default",
                              initial_resources=2000,
                              energy_needs=np.random.uniform(50, 200),
                              production_type="solar",
                              max_capacity=np.random.uniform(100, 300))
            # self.schedule.add(prosumer)
            
        # Create producers
        for i in range(self.num_producers):
            producer = EnergyProducerAgent(f"producer_{i}", self,
                                    persona="default",
                                    initial_resources=10000,
                                    production_type="renewable" if i % 2 == 0 else "fossil",
                                    max_capacity=np.random.uniform(500, 1000),
                                    production_costs=np.random.uniform(10, 30))
            # self.schedule.add(producer)
            
        # Create utilities
        utility_types = ["eco-friendly", "balanced", "greedy"]
        for i in range(self.num_utilities):
            utility = UtilityAgent(f"utility_{i}", self,
                            persona="default",
                            initial_resources=50000,
                            utility_type=utility_types[i % len(utility_types)],
                            renewable_quota=0.3)
            # self.schedule.add(utility)
            
        # Create regulator
        regulator = RegulatorAgent("regulator", self, persona="neutral")
        # self.schedule.add(regulator)
    
    def step(self):
        self.datacollector.collect(self)
        # self.schedule.step() 


In [33]:
model = EnergyMarketModel()

TypeError: ProsumerAgent.__init__() got an unexpected keyword argument 'production_type'